In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import sqlite3
pd.set_option('display.max_rows', None)

In [2]:
with open('IMDb _Top 250_ (Sorted by IMDb Rating Descending) - IMDb.html', 'r') as f:
    soup1 = BeautifulSoup(f)

with open('IMDb _Top 250_ (Sorted by IMDb Rating Descending)2 - IMDb.html', 'r') as f:
    soup2 = BeautifulSoup(f)
    
with open('IMDb _Top 250_ (Sorted by IMDb Rating Descending)3 - IMDb.html', 'r') as f:
    soup3 = BeautifulSoup(f)
    
with open('IMDb _Top 250_ (Sorted by IMDb Rating Descending)4 - IMDb.html', 'r') as f:
    soup4 = BeautifulSoup(f)
    
with open('IMDb _Top 250_ (Sorted by IMDb Rating Descending)5 - IMDb.html', 'r', encoding='utf-8') as f:
    soup5 = BeautifulSoup(f)
    
useful1 = soup1.findAll("div", attrs={'class':'lister-item-content'})
useful2 = soup2.findAll("div", attrs={'class':'lister-item-content'})
useful3 = soup3.findAll("div", attrs={'class':'lister-item-content'})
useful4 = soup3.findAll("div", attrs={'class':'lister-item-content'})
useful5 = soup3.findAll("div", attrs={'class':'lister-item-content'})

In [29]:
def ScrapeIMDB(html):
    rows = []
    for data in html:
        row = []
        rank = data.h3.span.getText()
        title = data.h3.a.getText()
        year = data.find('span', attrs={'class' : 'lister-item-year text-muted unbold'}).getText().strip(')').strip('(')
        attrs = data.p.findAll('span')
        info = []
        for i in attrs:
            if i.getText() == '|':
                continue
            else:
                info.append(i.getText().rstrip().strip('\n'))
        if len(info) == 1:
            rating = np.nan
            length = np.nan
            genres = info
        elif len(info) == 2:
            rating = np.nan
            length = info[0]
            genres = info[1]
        else:
            rating = info[0]
            length = info[1]
            genres = info[2]
        imdb_rating = data.div.div.strong.getText()
        try:
            metascore = data.find('div', attrs = {'class' : 'inline-block ratings-metascore'}).getText().strip('\n')[:2]
        except:
            metascore = np.nan
        cast_data = data.select('div > p')[2].findAll('a')
        cast = [a.getText() for a in cast_data]
        director = cast.pop(0)
        misc_data = cast_data = data.select('div > p')[3].findAll('span')
        misc = [span.getText() for span in misc_data]
        votes = misc[1]
        try:
            gross = misc[4]
        except:
            gross = np.nan
        row.extend([rank, title, year, rating, length, genres, imdb_rating, metascore, director, cast, votes, gross])
        rows.append(row)
    return rows

list1 = ScrapeIMDB(useful1)
list2 = ScrapeIMDB(useful2)
list3 = ScrapeIMDB(useful3)
list4 = ScrapeIMDB(useful4)
list5 = ScrapeIMDB(useful5)

list_final = list1 + list2 + list3 + list4 + list5

headers = ['Rank', 'Title', 'Release_Year', 'MPAA_Rating', 'Length_min', 'Genres', 'Score', 'Metascore',
          'Director', 'Cast', 'Score_Counts', 'Gross']

df = pd.DataFrame(list_final, columns = headers)

df['Rank'] = df['Rank'].str.rstrip('.')

df.at[58, 'Release_Year'] = 2017

df['Cast'] = df['Cast'].apply(', '.join)

df['Metascore'] = df['Metascore'].replace('10', '100')

# Change column types
df = df.astype({'Metascore' : 'float64', 'Score' : 'float64', 'Rank': 'int64',
               'Release_Year': 'int64'})

# Remove commas and change votes to int
df['Score_Counts'] = df['Score_Counts'].str.replace(',', '')
df['Score_Counts'] = pd.to_numeric(df['Score_Counts'])

# Removal of formatting and casting as int
df['Gross'] = df['Gross'].str.replace('$', '')
df['Gross'] = df['Gross'].str.replace('M', '')
df['Gross'] = pd.to_numeric(df['Gross'])

# Removal of text and casting as int
df['Length_min'] = df['Length_min'].str.split()
df['Length_min'] = df['Length_min'].map(lambda x: x[0])
df['Length_min'] = pd.to_numeric(df['Length_min'])


# Displays how much the IMDb Rating went up per million spent on the movie
df['Cost_Score_Ratio'] = df['Score']/df['Gross']

df['Type'] = 'Movie'

In [4]:
for col in df.columns:
    print(col, df[col].dtype)

Rank int64
Title object
Release_Year int64
MPAA_Rating object
Length_min int64
Genres object
Score float64
Metascore float64
Director object
Cast object
Score_Counts int64
Gross float64
Cost_Score_Ratio float64
Type object


In [5]:
with sqlite3.connect("Milestone5.db") as conn:
    cursor = conn.cursor()
    cursor.execute('DROP TABLE movies')
    cursor.execute('CREATE TABLE movies (Rank integer, Title object, Release_Year integer, MPAA_Rating text, Length_min integer, Genres text, Score integer, Metascore real, Director text, Cast text, Score_Counts integer, Gross real, Cost_Score_Ratio real, Type text)')
    # fill 'movies' table with the values from pandas dataframe
    df.to_sql('movies', conn, if_exists = 'replace', index = False)

In [31]:
df.to_csv('movies.csv', index = False)

In [30]:
df[df['Metascore']==100.0]

,Rank,Title,Release_Year,MPAA_Rating,Length_min,Genres,Score,Metascore,Director,Cast,Score_Counts,Gross,Cost_Score_Ratio,Type
1,2,The Godfather,1972,R,175,"Crime, Drama",9.2,100.0,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",1559060,134.97,0.068163,Movie
49,50,Casablanca,1942,PG,102,"Drama, Romance, War",8.5,100.0,Michael Curtiz,"Humphrey Bogart, Ingrid Bergman, Paul Henreid,...",506345,1.02,8.333333,Movie
77,78,Rear Window,1954,PG,112,"Mystery, Thriller",8.4,100.0,Alfred Hitchcock,"James Stewart, Grace Kelly, Wendell Corey, The...",428775,36.76,0.228509,Movie
111,112,Lawrence of Arabia,1962,Approved,228,"Adventure, Biography, Drama",8.3,100.0,David Lean,"Peter O'Toole, Alec Guinness, Anthony Quinn, J...",259392,44.82,0.185185,Movie
114,115,Vertigo,1958,PG,128,"Mystery, Romance, Thriller",8.3,100.0,Alfred Hitchcock,"James Stewart, Kim Novak, Barbara Bel Geddes, ...",348669,3.20,2.593750,Movie
119,120,Citizen Kane,1941,PG,119,"Drama, Mystery",8.3,100.0,Orson Welles,"Orson Welles, Joseph Cotten, Dorothy Comingore...",386344,1.59,5.220126,Movie
161,112,Lawrence of Arabia,1962,Approved,228,"Adventure, Biography, Drama",8.3,100.0,David Lean,"Peter O'Toole, Alec Guinness, Anthony Quinn, J...",259392,44.82,0.185185,Movie
164,115,Vertigo,1958,PG,128,"Mystery, Romance, Thriller",8.3,100.0,Alfred Hitchcock,"James Stewart, Kim Novak, Barbara Bel Geddes, ...",348669,3.20,2.593750,Movie
169,120,Citizen Kane,1941,PG,119,"Drama, Mystery",8.3,100.0,Orson Welles,"Orson Welles, Joseph Cotten, Dorothy Comingore...",386344,1.59,5.220126,Movie
211,112,Lawrence of Arabia,1962,Approved,228,"Adventure, Biography, Drama",8.3,100.0,David Lean,"Peter O'Toole, Alec Guinness, Anthony Quinn, J...",259392,44.82,0.185185,Movie
